# Data Input/Output for Signal Processing
#### Input: 
Takes in data file (text or image) and processes it into binary data. The first byte (8 bits) encodes the data format that is being transmitted - this is able to handle 3-bit errors

#### Output: 
Takes in binary data, depending on the format (encoded in the first 8 bits - either image or text), output is processed and saved to a suitable format. 

In [7]:
## Import modules
import numpy as np
from datetime import datetime

In [28]:
## read in a file - image or text - convert it to binary data


# get the filename
filename = "hello_world.txt"

# acceptable image formats for transmission
IM_FORMATS = ['png', 'jpeg']


# byte value representing the format of the data 
IM_CODE = 170 # 170 (uint8) = 10101010 (every even index is 1) in binary
TXT_CODE = 85 # 85 (uint8) = 01010101 (every odd index is 1) in binary
# able to handle 3-bit errors - needs to be robust as entire data packet depends on this value

# split into components - file type ext and title
fn = filename.split('.')
filetype = fn[-1].lower()
f_title = fn[0]




# since channel can be used to transcieve images and text, we encode the format as the first byte
# according to filetype
if filetype in IM_FORMATS:
    ft_code = IM_CODE 
else:
    ft_code = TXT_CODE 




# convert file contents to 8-bit integers (bytes)
send_bytes = np.fromfile(filename, dtype = "uint8")

# add filetype indicator as first value
send_bytes = np.insert(send_bytes, 0, ft_code)

# convert bytes into bits 
send_bits = np.unpackbits(send_bytes)

print(send_bits)



# produce list of binary data
send_data = list(send_bits)


binary_file = open('transmit.bin', 'wb')
send_bits.tofile(binary_file)
binary_file.close()




print(len(send_data))

[0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1
 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 1]
104


In [50]:
## read in (received) data and convert it out of binary 


IM_SAVE_FORM = '.png'


# receive the binary data (bits)
received_bits = np.fromfile('receive.bin', 'uint8')

# check if the sent bits match the received bits - ensures code is working (in absence of interference)
#print("All bits match: ", np.all(received_bits == send_bits))

# extract the format of the data (first 8 bits)
format_code = received_bits[:8]


fc_odd = sum(format_code[1::2]) # sum of bits at odd indices
fc_even = sum(format_code[0::2]) # sum of bits at even indices

# if more 1s at even indices, format = image
# Image = 10101010
# Text = 01010101
if fc_even > fc_odd:
    save_form = IM_SAVE_FORM
    
# otherwise it is text format
else:
    save_form = '.txt'

# remove the format code from the data
received_bits = received_bits[8:]


# get timestamp for current time
cur_date = datetime.now().strftime("%Y_%m_%d_%I-%M-%S_%p")

# convert bits to bytes
received_bytes = np.packbits(received_bits)
print(received_bytes)

# save to output file
#received_bytes.tofile("rec_"+ cur_date + save_form) 
received_bytes.tofile("rec_"+ save_form)

[  0 128   0 128   0 128   0 128   0 128 128   0 128   0   0   0   0 128
 128   0   0 128   0 128   0 128 128   0 128 128   0   0   0 128 128   0
 128 128   0   0   0 128 128   0 128 128 128 128   0   0 128   0   0   0
   0   0   0 128 128 128   0 128 128 128   0 128 128   0 128 128 128 128
   0 128 128 128   0   0 128   0   0 128 128   0 128 128   0   0   0 128
 128   0   0 128   0   0   0   0 128   0   0   0   0]
